<a href="https://colab.research.google.com/github/ste-corizzato/NLP-Project/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Training**

In [ ]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Config
from transformers import get_linear_schedule_with_warmup

import torch
from torch.optim import AdamW #optimizer
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split, RandomSampler, SequentialSampler

from google.colab import drive
import pandas as pd
import random
import os

from tqdm.auto import tqdm
import datetime
import time

##Load dataset

In [ ]:
gdrive_dir = '/content/gdrive/'
#data_dir = os.path.join(gdrive_dir, "My Drive","NLP","Project(P2)","df_medium_final.csv") #20'000 ricette (MEDIO)
data_dir = os.path.join(gdrive_dir, "My Drive","NLP","Project(P2)","df_lil_final.csv") #10'000 ricette (PICCOLO)

In [ ]:
drive.mount(gdrive_dir)

Mounted at /content/gdrive/


In [ ]:
df_final = pd.read_csv(data_dir)

In [ ]:
df_final.head()

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
model_name = "gpt2" #"gpt2-medium"
model_save_path  = '/content/gdrive/My Drive/NLP/Project(P2)/model-p-10mila'
#model_save_path  = '/content/gdrive/My Drive/NLP/Project(P2)/model-p-20mila'
#model_save_path  = '/content/gdrive/My Drive/NLP/Project(P2)/model-m-10mila'

##Test the model before fine-tuning

In [ ]:
configuration = GPT2Config.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, config=configuration)

tokenizer = GPT2TokenizerFast.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
df_final.iloc[820]['RecipeIngredientParts'], df_final.iloc[820]['RecipeInstructions']

("['whole corn' 'sour cream' 'butter' 'eggs']",
 '[\'Mix all ingredients in a buttered 8" square baking pan (glass pans work best).\'\n \'Stir until all ingredients are mixed well.\'\n \'Bake 1 hour at 350 degrees F, or 15-18 minutes in a microwave on full power.\'\n \'Refrigerate leftovers.\']')

In [ ]:
input_sequence = "Ingredients: whole corn, sour cream, butter, eggs"
input_ids = tokenizer.encode(input_sequence, return_tensors='pt')

In [ ]:
model = model.to(device)
#combine both sampling techniques
sample_outputs = model.generate(input_ids.to(device),
                              do_sample = True, max_length = 120,
                              top_k = 50, top_p = 0.85,
                              num_return_sequences = 3)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('-----------------------------------')

Output:
----------------------------------------------------------------------------------------------------
0: Ingredients: whole corn, sour cream, butter, eggs, vanilla extract

Serves 4

Ingredients: whole wheat, bread flour, water, milk

Serves 4

Ingredients: whole wheat, bread flour, water, milk

Serves 4

Ingredients: whole wheat, bread flour, water, milk

Serves 4

Ingredients: whole wheat, bread flour, water, milk

Serves 4

Ingredients: whole wheat, bread flour, water, milk

Serves 4

Ingredients: whole wheat, bread flour, water
-----------------------------------
1: Ingredients: whole corn, sour cream, butter, eggs

Directions: In a small bowl, combine all of the ingredients in a medium saucepan. Cook until the sugar and flour mix together, about 2 minutes.

Preheat oven to 350 degrees.

Spread out a half-dozen sheets of parchment paper or parchment paper with the filling.

Sprinkle the mixture over the top of the paper, using a toothpick or an ice cream scoop.

Place on a b

##Funzioni Utily

In [ ]:
def form_string(ingredient,instruction, id):
  s = f"<|startoftext|>Ingredients: {ingredient.strip()}. " \
      f"Instructions: {instruction.strip()}<|endoftext|>"
  return s

In [ ]:
def extract_string(recipe):
    str = recipe.replace('<|startoftext|>', '').replace('<|endoftext|>', '')
    inst_pos = str.find('Instructions: ')
    ingredients = str[len('Ingredients: '): inst_pos-1]
    instructions = str[inst_pos+len('Instructions: '):]
    return ingredients, instructions


In [ ]:
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [ ]:
def infer(prompt):
    input = f"<|startoftext|>Ingredients: {prompt.strip()}"
    input = tokenizer(input, return_tensors="pt")
    input_ids      = input["input_ids"]
    attention_mask = input["attention_mask"]

    output = model.generate(input_ids.to(device),
                            attention_mask=attention_mask.to(device),
                            max_new_tokens=max_length,
                            # temperature = 0.5,
                            do_sample = True, top_k = 50, top_p = 0.85)
                            # num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    return output

##Tokenize the dataset

In [ ]:
df_final[:1]

,Unnamed: 0.1,Unnamed: 0,RecipeId,RecipeIngredientParts,AggregatedRating,ReviewCount,RecipeInstructions
0,0,41924,45809.0,"boneless chicken breasts, olive oil, garlic cl...",5.0,3063.0,Editor's Note: Named Bourbon Chicken because ...


In [ ]:
data = df_final.apply(lambda x:form_string(
    x['RecipeIngredientParts'], x['RecipeInstructions']) axis=1).to_list()   #, x['RecipeId']), axis=1).to_list()
data[0]

"<|startoftext|>Ingredients: boneless chicken breasts, olive oil, garlic clove, ginger, crushed red pepper flakes, light brown sugar, ketchup, cider vinegar, water, soy sauce. Instructions: Editor's Note:  Named Bourbon Chicken because it was supposedly created by a Chinese cook who worked in a restaurant on Bourbon Street., Heat oil in a large skillet., Add chicken pieces and cook until lightly browned., Remove chicken., Add remaining ingredients, heating over medium Heat until well mixed and dissolved., Add chicken and bring to a hard boil., Reduce heat and simmer for 20 minutes., Serve over hot rice and ENJOY.<|endoftext|>"

In [ ]:
tokenizer = GPT2TokenizerFast.from_pretrained(model_name,
                                              bos_token='<|startoftext|>',
                                              eos_token='<|endoftext|>',
                                              unk_token='<|unknown|>',
                                              pad_token='<|pad|>'
                                             )

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
max_length = 180

# standard PyTorch approach of loading data in using a Dataset class.
class RecipeDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.input_ids = []
        self.attn_masks = []
        self.origin_ingredients = []
        self.origin_instructions = []

        for recipe in data:
            encodings = tokenizer.encode_plus(recipe,
                                              truncation=True,
                                              padding='max_length',
                                              max_length=max_length,
                                              return_tensors='pt'
                                             )
            self.input_ids.append(torch.squeeze(encodings['input_ids'],0))
            self.attn_masks.append(torch.squeeze(encodings['attention_mask'],0))
            ingredients, instructions = extract_string(recipe)
            self.origin_ingredients.append(ingredients)
            self.origin_instructions.append(instructions)

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.input_ids[idx], self.attn_masks[idx], self.origin_ingredients[idx], self.origin_instructions[idx]

In [ ]:
dataset = RecipeDataset(data, tokenizer)

print(f"input_ids: {dataset[0][0]} attn_masks: {dataset[0][1]}")

input_ids: tensor([50257, 41222,    25,  5351,  5321,  9015, 17515,    11, 19450,  3056,
           11, 17644,   537,   659,    11, 25072,    11, 18577,  2266, 13385,
        42330,    11,  1657,  7586,  7543,    11,   479, 47132,    11, 36930,
        26600,    11,  1660,    11, 17797, 10746,    13, 27759,    25, 12058,
          338,  5740,    25,   220, 34441, 41638, 16405,   780,   340,   373,
        13519,  2727,   416,   257,  3999,  4255,   508,  3111,   287,   257,
         7072,   319, 41638,  3530,  1539, 12308,  3056,   287,   257,  1588,
        41306,  1539,  3060,  9015,  5207,   290,  4255,  1566, 15376,  7586,
          276,  1539, 17220,  9015,  1539,  3060,  5637,  9391,    11, 16930,
          625,  7090, 12308,  1566,   880,  7668,   290, 26306,  1539,  3060,
         9015,   290,  2222,   284,   257,  1327, 20667,  1539, 44048,  4894,
          290, 32857,   329,  1160,  2431,  1539, 35557,   625,  3024, 11464,
          290, 12964,    41, 21414,    13, 50256, 502

In [ ]:
dataset[0][3]

"Editor's Note:  Named Bourbon Chicken because it was supposedly created by a Chinese cook who worked in a restaurant on Bourbon Street., Heat oil in a large skillet., Add chicken pieces and cook until lightly browned., Remove chicken., Add remaining ingredients, heating over medium Heat until well mixed and dissolved., Add chicken and bring to a hard boil., Reduce heat and simmer for 20 minutes., Serve over hot rice and ENJOY."

In [ ]:
batch_size = 8

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


train_dataloader = DataLoader(
            train_dataset,   # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

## Finetuning

In [ ]:
configuration = GPT2Config.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, config=configuration)
model = model.to(device)

print(f"Weight shape {model.transformer.wte.weight.shape}")

model.resize_token_embeddings(len(tokenizer))
print(f"Number of tokens: {len(tokenizer)}")

seed_val = 42

random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

Weight shape torch.Size([50257, 768])
Number of tokens: 50260


In [ ]:
epochs = 3
learning_rate = 2e-5
warmup_steps = 1e2

# The epsilon parameter eps = 1e-8 is “a very small number to prevent any division by zero in the implementation”
epsilon = 1e-8
optim = AdamW(model.parameters(), lr = learning_rate, eps = epsilon)

In [ ]:
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optim,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)

In [ ]:
total_steps

3375

In [ ]:
len(train_dataloader)

1125

In [ ]:
total_t0 = time.time()

training_stats = []

for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()  # `train` just changes the *mode* (train vs. eval), it doesn't *perform* the training.

    for step, batch in enumerate(train_dataloader):     # step from enumerate() = number of batches

        b_input_ids = batch[0].to(device)   # tokens (of multiple documents in a batch)
        b_labels    = batch[0].to(device)
        b_masks     = batch[1].to(device)   # mask of [1] for a real word, [0] for a pad

        model.zero_grad()
        outputs = model(  input_ids = b_input_ids,
                          labels = b_labels,
                          attention_mask = b_masks,
                          token_type_ids = None
                        )

        loss = outputs[0]

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % 200 == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

        loss.backward()
        optim.step()
        scheduler.step()

    # Calculate the average loss over all the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))


    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():

            outputs  = model(input_ids = b_input_ids,
                             attention_mask = b_masks,
                             labels = b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 3 ========
Training...
  Batch   200  of  1,125. Loss: 2.9421465396881104.   Elapsed: 0:01:18.
  Batch   400  of  1,125. Loss: 2.1251614093780518.   Elapsed: 0:02:40.
  Batch   600  of  1,125. Loss: 1.9413484334945679.   Elapsed: 0:04:02.
  Batch   800  of  1,125. Loss: 1.682545781135559.   Elapsed: 0:05:26.
  Batch 1,000  of  1,125. Loss: 1.912654995918274.   Elapsed: 0:06:49.

  Average training loss: 2.33
  Training epoch took: 0:07:42

Running Validation...
  Validation Loss: 1.70
  Validation took: 0:00:17

======== Epoch 2 / 3 ========
Training...
  Batch   200  of  1,125. Loss: 2.0031659603118896.   Elapsed: 0:01:25.
  Batch   400  of  1,125. Loss: 1.5143709182739258.   Elapsed: 0:02:50.
  Batch   600  of  1,125. Loss: 1.630144715309143.   Elapsed: 0:04:15.
  Batch   800  of  1,125. Loss: 1.613868236541748.   Elapsed: 0:05:40.
  Batch 1,000  of  1,125. Loss: 1.5449025630950928.   Elapsed: 0:07:05.

  Average training loss: 1.71
  Training epoch took: 0:07:58


In [ ]:
df_stats = pd.DataFrame(data=training_stats)

df_stats = df_stats.set_index('epoch')
df_stats

,Training Loss,Valid. Loss,Training Time,Validation Time
epoch,,,,
1,2.333309,1.700797,0:07:42,0:00:17
2,1.708053,1.662699,0:07:58,0:00:17
3,1.666428,1.650435,0:07:58,0:00:17


## Saving the model

In [ ]:
print("Saving model to %s" % model_save_path)

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

Saving model to /content/gdrive/My Drive/NLP/Project(P2)/model-p-10mila


('/content/gdrive/My Drive/NLP/Project(P2)/model-p-10mila/tokenizer_config.json',
 '/content/gdrive/My Drive/NLP/Project(P2)/model-p-10mila/special_tokens_map.json',
 '/content/gdrive/My Drive/NLP/Project(P2)/model-p-10mila/vocab.json',
 '/content/gdrive/My Drive/NLP/Project(P2)/model-p-10mila/merges.txt',
 '/content/gdrive/My Drive/NLP/Project(P2)/model-p-10mila/added_tokens.json',
 '/content/gdrive/My Drive/NLP/Project(P2)/model-p-10mila/tokenizer.json')

In [ ]:
model = GPT2LMHeadModel.from_pretrained(model_save_path)
tokenizer = GPT2TokenizerFast.from_pretrained(model_save_path)
model.to(device)

##Testing the model

In [ ]:
# model = GPT2LMHeadModel.from_pretrained(model_save_path)
# tokenizer = GPT2TokenizerFast.from_pretrained(model_save_path)
# model.to(device)
a = infer("eggs, mushroom, butter, sugar")
a
#print(infer("eggs, mushroom, butter, sugar"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Ingredients: eggs, mushroom, butter, sugar, salt, ground pepper, sour cream. Instructions: Preheat oven to 350 degrees., Melt butter and sugars in heavy skillet., Add mushrooms and cook for a few minutes stirring occasionally., Add the butter mixture to the hot mushroom mixture and cook stirring occasionally, stirring in a little water when stirring., Return to stove and stir until smooth., Return the hot mixture to the stove and cook stirring constantly until hot and bubbly., Pour the hot mix over the mushrooms and serve with crackers, crackers, ice cream.'

## Medium 10'000


In [ ]:
model_name = "gpt2-medium"
model_save_path  = '/content/gdrive/My Drive/NLP/Project(P2)/model-m-10mila'

In [ ]:
tokenizer_medium = GPT2TokenizerFast.from_pretrained(model_name,
                                              bos_token='<|startoftext|>',
                                              eos_token='<|endoftext|>',
                                              unk_token='<|unknown|>',
                                              pad_token='<|pad|>'
                                             )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

In [ ]:
max_length = 180

# standard PyTorch approach of loading data in using a Dataset class.
class RecipeDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.input_ids = []
        self.attn_masks = []
        self.origin_ingredients = []
        self.origin_instructions = []

        for recipe in data:
            encodings = tokenizer.encode_plus(recipe,
                                              truncation=True,
                                              padding='max_length',
                                              max_length=max_length,
                                              return_tensors='pt'
                                             )
            self.input_ids.append(torch.squeeze(encodings['input_ids'],0))
            self.attn_masks.append(torch.squeeze(encodings['attention_mask'],0))
            ingredients, instructions = extract_string(recipe)
            self.origin_ingredients.append(ingredients)
            self.origin_instructions.append(instructions)

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.input_ids[idx], self.attn_masks[idx], self.origin_ingredients[idx], self.origin_instructions[idx]

In [ ]:
dataset = RecipeDataset(data, tokenizer_medium)

In [ ]:
batch_size = 8 #2 o 4 ??? forse problemi di memoria ma non so


train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


train_dataloader = DataLoader(
            train_dataset,   # The training samples.
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size # Trains with this batch size.
        )

validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
configuration = GPT2Config.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, config=configuration)
model = model.to(device)

print(f"Weight shape {model.transformer.wte.weight.shape}")
model.resize_token_embeddings(len(tokenizer))
print(f"Number of tokens: {len(tokenizer)}")

seed_val = 42

random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Weight shape torch.Size([50257, 1024])
Number of tokens: 50260


In [ ]:
epochs = 3
learning_rate = 2e-5
warmup_steps = 1e2

epsilon = 1e-8
optim = AdamW(model.parameters(), lr = learning_rate, eps = epsilon)

In [ ]:
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optim,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)

In [ ]:
total_steps

3375

In [ ]:
len(train_dataloader)

1125

In [ ]:
total_t0 = time.time()

training_stats = []

for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()  # `train` just changes the *mode* (train vs. eval), it doesn't *perform* the training.

    for step, batch in enumerate(train_dataloader):     # step from enumerate() = number of batches

        b_input_ids = batch[0].to(device)   # tokens (of multiple documents in a batch)
        b_labels    = batch[0].to(device)
        b_masks     = batch[1].to(device)   # mask of [1] for a real word, [0] for a pad

        model.zero_grad()
        outputs = model(  input_ids = b_input_ids,
                          labels = b_labels,
                          attention_mask = b_masks,
                          token_type_ids = None
                        )

        loss = outputs[0]

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % 200 == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

        loss.backward()
        optim.step()
        scheduler.step()

    # Calculate the average loss over all the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))


    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():

            outputs  = model(input_ids = b_input_ids,
                             attention_mask = b_masks,
                             labels = b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 3 ========
Training...
  Batch   200  of  1,125. Loss: 1.8483610153198242.   Elapsed: 0:03:46.
  Batch   400  of  1,125. Loss: 1.7977306842803955.   Elapsed: 0:07:36.
  Batch   600  of  1,125. Loss: 1.175990343093872.   Elapsed: 0:11:27.
  Batch   800  of  1,125. Loss: 2.0177483558654785.   Elapsed: 0:15:17.
  Batch 1,000  of  1,125. Loss: 1.1443268060684204.   Elapsed: 0:19:07.

  Average training loss: 2.01
  Training epoch took: 0:21:30

Running Validation...
  Validation Loss: 1.49
  Validation took: 0:00:43

======== Epoch 2 / 3 ========
Training...
  Batch   200  of  1,125. Loss: 1.49465811252594.   Elapsed: 0:03:50.
  Batch   400  of  1,125. Loss: 1.64085853099823.   Elapsed: 0:07:41.
  Batch   600  of  1,125. Loss: 1.7630529403686523.   Elapsed: 0:11:31.
  Batch   800  of  1,125. Loss: 1.2818735837936401.   Elapsed: 0:15:22.
  Batch 1,000  of  1,125. Loss: 1.422047734260559.   Elapsed: 0:19:12.

  Average training loss: 1.49
  Training epoch took: 0:21:35

R

In [ ]:
df_stats = pd.DataFrame(data=training_stats)

df_stats = df_stats.set_index('epoch')
df_stats

,Training Loss,Valid. Loss,Training Time,Validation Time
epoch,,,,
1,2.011273,1.485370,0:21:30,0:00:43
2,1.490836,1.455428,0:21:35,0:00:43
3,1.445643,1.446634,0:21:35,0:00:43


In [ ]:
print("Saving model to %s" % model_save_path)

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

Saving model to /content/gdrive/My Drive/NLP/Project(P2)/model-check


('/content/gdrive/My Drive/NLP/Project(P2)/model-check/tokenizer_config.json',
 '/content/gdrive/My Drive/NLP/Project(P2)/model-check/special_tokens_map.json',
 '/content/gdrive/My Drive/NLP/Project(P2)/model-check/vocab.json',
 '/content/gdrive/My Drive/NLP/Project(P2)/model-check/merges.txt',
 '/content/gdrive/My Drive/NLP/Project(P2)/model-check/added_tokens.json',
 '/content/gdrive/My Drive/NLP/Project(P2)/model-check/tokenizer.json')

## Small 20'000

In [ ]:
data_dir = os.path.join(gdrive_dir, "My Drive","NLP","Project(P2)","df_medium_final.csv") #20'000 ricette (MEDIO)

In [ ]:
df_final = pd.read_csv(data_dir)

In [ ]:
model_name = "gpt2"
model_save_path  = '/content/gdrive/My Drive/NLP/Project(P2)/model-p-20mila'

In [ ]:
data = df_final.apply(lambda x:form_string(
    x['RecipeIngredientParts'], x['RecipeInstructions'], x['RecipeId']), axis=1).to_list()
data[0]

"<|startoftext|>Ingredients: boneless chicken breasts, olive oil, garlic clove, ginger, crushed red pepper flakes, light brown sugar, ketchup, cider vinegar, water, soy sauce. Instructions: Editor's Note:  Named Bourbon Chicken because it was supposedly created by a Chinese cook who worked in a restaurant on Bourbon Street., Heat oil in a large skillet., Add chicken pieces and cook until lightly browned., Remove chicken., Add remaining ingredients, heating over medium Heat until well mixed and dissolved., Add chicken and bring to a hard boil., Reduce heat and simmer for 20 minutes., Serve over hot rice and ENJOY.<|endoftext|>"

In [ ]:
max_length = 180

class RecipeDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.input_ids = []
        self.attn_masks = []
        self.origin_ingredients = []
        self.origin_instructions = []

        for recipe in data:
            encodings = tokenizer.encode_plus(recipe,
                                              truncation=True,
                                              padding='max_length',
                                              max_length=max_length,
                                              return_tensors='pt'
                                             )
            self.input_ids.append(torch.squeeze(encodings['input_ids'],0))
            self.attn_masks.append(torch.squeeze(encodings['attention_mask'],0))
            ingredients, instructions = extract_string(recipe)
            self.origin_ingredients.append(ingredients)
            self.origin_instructions.append(instructions)

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.input_ids[idx], self.attn_masks[idx], self.origin_ingredients[idx], self.origin_instructions[idx]

In [ ]:
dataset = RecipeDataset(data, tokenizer)

In [ ]:
batch_size = 8


train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(
            train_dataset,   # The training samples.
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

In [ ]:
configuration = GPT2Config.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, config=configuration)
model = model.to(device)

print(f"Weight shape {model.transformer.wte.weight.shape}")
model.resize_token_embeddings(len(tokenizer))
print(f"Number of tokens: {len(tokenizer)}")

seed_val = 42

random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

Weight shape torch.Size([50257, 768])
Number of tokens: 50260


In [ ]:
epochs = 3
learning_rate = 2e-5
warmup_steps = 1e2

epsilon = 1e-8
optim = AdamW(model.parameters(), lr = learning_rate, eps = epsilon)

In [ ]:
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optim,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)

In [ ]:
total_steps

6750

In [ ]:
len(train_dataloader)

2250

In [ ]:
total_t0 = time.time()

training_stats = []

for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()  # `train` just changes the *mode* (train vs. eval), it doesn't *perform* the training.

    for step, batch in enumerate(train_dataloader):     # step from enumerate() = number of batches

        b_input_ids = batch[0].to(device)   # tokens (of multiple documents in a batch)
        b_labels    = batch[0].to(device)
        b_masks     = batch[1].to(device)   # mask of [1] for a real word, [0] for a pad

        model.zero_grad()
        outputs = model(  input_ids = b_input_ids,
                          labels = b_labels,
                          attention_mask = b_masks,
                          token_type_ids = None
                        )

        loss = outputs[0]

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % 200 == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

        loss.backward()
        optim.step()
        scheduler.step()

    # Calculate the average loss over all the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))


    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():

            outputs  = model(input_ids = b_input_ids,
                             attention_mask = b_masks,
                             labels = b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 3 ========
Training...
  Batch   200  of  2,250. Loss: 2.802865743637085.   Elapsed: 0:01:25.
  Batch   400  of  2,250. Loss: 1.777146816253662.   Elapsed: 0:02:48.
  Batch   600  of  2,250. Loss: 1.857603907585144.   Elapsed: 0:04:12.
  Batch   800  of  2,250. Loss: 1.7185263633728027.   Elapsed: 0:05:35.
  Batch 1,000  of  2,250. Loss: 1.6625311374664307.   Elapsed: 0:06:59.
  Batch 1,200  of  2,250. Loss: 1.733724594116211.   Elapsed: 0:08:22.
  Batch 1,400  of  2,250. Loss: 1.6650112867355347.   Elapsed: 0:09:46.
  Batch 1,600  of  2,250. Loss: 1.3545726537704468.   Elapsed: 0:11:10.
  Batch 1,800  of  2,250. Loss: 1.7912261486053467.   Elapsed: 0:12:33.
  Batch 2,000  of  2,250. Loss: 1.3850624561309814.   Elapsed: 0:13:57.
  Batch 2,200  of  2,250. Loss: 1.9588432312011719.   Elapsed: 0:15:20.

  Average training loss: 2.02
  Training epoch took: 0:15:41

Running Validation...
  Validation Loss: 1.61
  Validation took: 0:00:32

======== Epoch 2 / 3 ========
Tr

In [ ]:
df_stats = pd.DataFrame(data=training_stats)

df_stats = df_stats.set_index('epoch')
df_stats

,Training Loss,Valid. Loss,Training Time,Validation Time
epoch,,,,
1,2.020930,1.605746,0:15:41,0:00:32
2,1.640415,1.565656,0:15:40,0:00:32
3,1.597094,1.552800,0:15:41,0:00:32


In [ ]:
print("Saving model to %s" % model_save_path)

Saving model to /content/gdrive/My Drive/NLP/Project(P2)/model-p-20mila


In [ ]:
print("Saving model to %s" % model_save_path)

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

Saving model to /content/gdrive/My Drive/NLP/Project(P2)/model-p-20mila


('/content/gdrive/My Drive/NLP/Project(P2)/model-p-20mila/tokenizer_config.json',
 '/content/gdrive/My Drive/NLP/Project(P2)/model-p-20mila/special_tokens_map.json',
 '/content/gdrive/My Drive/NLP/Project(P2)/model-p-20mila/vocab.json',
 '/content/gdrive/My Drive/NLP/Project(P2)/model-p-20mila/merges.txt',
 '/content/gdrive/My Drive/NLP/Project(P2)/model-p-20mila/added_tokens.json',
 '/content/gdrive/My Drive/NLP/Project(P2)/model-p-20mila/tokenizer.json')